In [25]:
import pandas as pd
import os
import time
import requests
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv(r'C:\Users\clint\Desktop\Geocoding_Documentation\.env')

df = pd.read_csv(r'C:\Users\clint\Desktop\Geocoding_Documentation\Notebooks\9.csv')
df

,clean_line1,clean_line2,line3,city,zip_code,label,phone,year,major_city,state,...,min_distance_miles_unconstrained,min_distance_sources_unconstrained,Mid_Lat,Mid_Long,Match_Comments,Manual_Match,Manual_Lat,Manual_Long,Final_Lat,Final_Long
0,"Coalville , 84017 Holiday Hills ( 66 )",435-336-4421 I-80 Exit 162 ( UT 280 ),MO,Coalville,84017,Holiday Hills ( 66 ),435-336-4421,2006,Coalville,UT,...,0.071012,Webscraped vs Yelp (Phone),40.910948,-111.404529,NaN,NaN,NaN,NaN,40.910948,-111.404529
1,"Coalville , 84017 Hills ( 66 )",435-336-4421 I-80 Holiday Exit 162 ( UT 280 ),M <U+25A1>,Coalville,84017,Hills ( 66 ),435-336-4421,2007,Coalville,UT,...,0.071012,Webscraped vs Yelp (Phone),40.910948,-111.404529,NaN,NaN,NaN,NaN,40.910948,-111.404529
2,"D Coalville , 84017 Holiday Hills ( 66 ) )",4 435-336-4421 I-80 Exit 162 ( UT 280 ),M <U+2610> <U+2610> <U+2610>,Coalville,84017,Holiday Hills ( 66 ) ),435-336-4421,2008,Coalville,UT,...,0.071012,Webscraped vs Yelp (Phone),40.910948,-111.404529,NaN,NaN,NaN,NaN,40.910948,-111.404529
3,D Coalville ( 84017 ) Holiday Hills ( 66 ),435-336-4421 I-80 Exit 162 ( UT 280 ),M <U+25A1> <U+2610>,Coalville,84017,Holiday Hills ( 66 ),435-336-4421,2014,Coalville,UT,...,0.071012,Webscraped vs Yelp (Phone),40.910948,-111.404529,NaN,NaN,NaN,NaN,40.910948,-111.404529
4,D Coalville ( 84017 ) Holiday Hills ( 66 ),4 435-336-4421 I-80 Exit 162 ( UT 280 ),D M <U+25A1> D D,Coalville,84017,Holiday Hills ( 66 ),435-336-4421,2015,Coalville,UT,...,0.071012,Webscraped vs Yelp (Phone),40.910948,-111.404529,NaN,NaN,NaN,NaN,40.910948,-111.404529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2328,Corning ( 96021 ) Love's Travel Stop # 410,2 530-824-8767 I-5 Exit 630 ( South Ave ),24 XL 24 24,Corning,96021,Love's Travel Stop # 410,530-824-8767,2016,Corning,CA,...,0.026483,Webscraped vs YellowPages (Phone),39.906627,-122.195031,NaN,NaN,NaN,NaN,39.906627,-122.195031
2329,", 96021 Flying J Travel Plaza # 0510154",CCorning 2530-824-8770 I-5 Exit 630 ( South Ave ),XL 24 45,NaN,96021,Flying J Travel Plaza # 0510154,530-824-8770,2007,Corning,CA,...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.646968,-122.528414
2330,"Corning , 96021 Flying J Travel Plaza # 0510154",2 530-824-8770 I-5 Exit 630 ( South Ave ),124 HRS XL 24 HRS,Corning,96021,Flying J Travel Plaza # 0510154,530-824-8770,2008,Corning,CA,...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.646968,-122.528414
2331,A Grenada ( 96038 ) 3 J's Food Mart ( 76 ),530-436-2208 I-5 Exit 766 ( CA A12 E ),<U+2610><U+2610> XL <U+2610>,Grenada,96038,3 J's Food Mart ( 76 ),530-436-2208,2014,Grenada,CA,...,0.021885,Yelp (Phone) vs YellowPages (Phone),41.646968,-122.528414,NaN,NaN,NaN,NaN,41.646968,-122.528414


In [26]:
# Check available columns and sample data
print("Available columns:")
print(df.columns.tolist())
print(f"\nDataframe shape: {df.shape}")

# Look for city, zip_code, and state columns
relevant_cols = [col for col in df.columns if any(keyword in col.lower() for keyword in ['city', 'zip', 'state', 'address'])]
print(f"\nPotentially relevant columns: {relevant_cols}")

# Show a sample of the data
print("\nSample data:")
df.head()

Available columns:
['clean_line1', 'clean_line2', 'line3', 'city', 'zip_code', 'label', 'phone', 'year', 'major_city', 'state', 'chain', 'address_standardized_ON_parenthesis', 'address_standardized_OFF_parenthesis', 'Address_Type', 'Exit_Number', 'Exit_From_Address', 'Exit_From_Label', 'Flagged', 'Flag_Reason', 'Is_Unclear_OCR', 'Main_Road', 'Secondary_Road', 'Exit_Number_2', 'Exit_Number_3', 'Tertiary_Road', 'phone_scraped_matches_row_ids', 'ZIP_scraped_matches_row_ids', 'City_scraped_matches_row_ids', 'Exit_scraped_matches_row_ids', 'State_scraped_matches_row_ids', 'Road_scraped_matches_row_ids', 'Chain_scraped_matches_row_ids', 'Label_scraped_matches_row_ids', 'all_matches', 'Phone_scraped_matches_row_ids', 'has_phone_match', 'ZIP_scraped_matches_row_ids_parsed', 'State_scraped_matches_row_ids_parsed', 'City_scraped_matches_row_ids_parsed', 'Exit_scraped_matches_row_ids_parsed', 'Road_scraped_matches_row_ids_parsed', 'Label_scraped_matches_row_ids_parsed', 'Chain_scraped_matches_row

,clean_line1,clean_line2,line3,city,zip_code,label,phone,year,major_city,state,...,min_distance_miles_unconstrained,min_distance_sources_unconstrained,Mid_Lat,Mid_Long,Match_Comments,Manual_Match,Manual_Lat,Manual_Long,Final_Lat,Final_Long
0,"Coalville , 84017 Holiday Hills ( 66 )",435-336-4421 I-80 Exit 162 ( UT 280 ),MO,Coalville,84017,Holiday Hills ( 66 ),435-336-4421,2006,Coalville,UT,...,0.071012,Webscraped vs Yelp (Phone),40.910948,-111.404529,NaN,NaN,NaN,NaN,40.910948,-111.404529
1,"Coalville , 84017 Hills ( 66 )",435-336-4421 I-80 Holiday Exit 162 ( UT 280 ),M <U+25A1>,Coalville,84017,Hills ( 66 ),435-336-4421,2007,Coalville,UT,...,0.071012,Webscraped vs Yelp (Phone),40.910948,-111.404529,NaN,NaN,NaN,NaN,40.910948,-111.404529
2,"D Coalville , 84017 Holiday Hills ( 66 ) )",4 435-336-4421 I-80 Exit 162 ( UT 280 ),M <U+2610> <U+2610> <U+2610>,Coalville,84017,Holiday Hills ( 66 ) ),435-336-4421,2008,Coalville,UT,...,0.071012,Webscraped vs Yelp (Phone),40.910948,-111.404529,NaN,NaN,NaN,NaN,40.910948,-111.404529
3,D Coalville ( 84017 ) Holiday Hills ( 66 ),435-336-4421 I-80 Exit 162 ( UT 280 ),M <U+25A1> <U+2610>,Coalville,84017,Holiday Hills ( 66 ),435-336-4421,2014,Coalville,UT,...,0.071012,Webscraped vs Yelp (Phone),40.910948,-111.404529,NaN,NaN,NaN,NaN,40.910948,-111.404529
4,D Coalville ( 84017 ) Holiday Hills ( 66 ),4 435-336-4421 I-80 Exit 162 ( UT 280 ),D M <U+25A1> D D,Coalville,84017,Holiday Hills ( 66 ),435-336-4421,2015,Coalville,UT,...,0.071012,Webscraped vs Yelp (Phone),40.910948,-111.404529,NaN,NaN,NaN,NaN,40.910948,-111.404529


In [27]:
# Examine 'Proper' addresses and their associated geographic data
proper_addresses = df[df['Address_Type'] == 'Proper']
print(f"Found {len(proper_addresses)} 'Proper' addresses")

# Look at the key columns for address construction
address_cols = ['address_standardized_OFF_parenthesis', 'city', 'zip_code', 'state']
print(f"\nSample of address components for 'Proper' addresses:")
sample_data = proper_addresses[address_cols].head(10)
for idx, row in sample_data.iterrows():
    print(f"Row {idx}:")
    for col in address_cols:
        print(f"  {col}: {row[col]}")
    print()
    
# Check for missing data in key columns
print("Missing data counts in key columns:")
for col in address_cols:
    missing_count = proper_addresses[col].isna().sum()
    print(f"  {col}: {missing_count} missing ({missing_count/len(proper_addresses)*100:.1f}%)")

Found 297 'Proper' addresses

Sample of address components for 'Proper' addresses:
Row 8:
  address_standardized_OFF_parenthesis: 1590 US 40 S
  city: Heber City
  zip_code: 84032
  state: UT

Row 9:
  address_standardized_OFF_parenthesis: 1590 US 40 S
  city: Heber City
  zip_code: 84032
  state: UT

Row 10:
  address_standardized_OFF_parenthesis: 1590 US 40 S
  city: Heber City
  zip_code: 84032
  state: UT

Row 11:
  address_standardized_OFF_parenthesis: 1590 US 40 S
  city: Heber City
  zip_code: 84032
  state: UT

Row 27:
  address_standardized_OFF_parenthesis: 1128 US 40
  city: Roosevelt
  zip_code: 84066
  state: UT

Row 28:
  address_standardized_OFF_parenthesis: 1128 US 40
  city: Roosevelt
  zip_code: 84066
  state: UT

Row 29:
  address_standardized_OFF_parenthesis: 1128 US 40
  city: Roosevelt
  zip_code: 84066
  state: UT

Row 42:
  address_standardized_OFF_parenthesis: 1355 US 40 E
  city: Vernal
  zip_code: 84078
  state: UT

Row 43:
  address_standardized_OFF_parenthes

In [28]:
# Improved geocoding function that combines address components
def create_comprehensive_address(street_address, city, zip_code, state):
    """
    Create a comprehensive address string by combining street address, city, zip code, and state
    This follows standard US address formatting for better geocoding results
    """
    components = []
    
    # Add street address if available
    if pd.notna(street_address) and str(street_address).strip():
        components.append(str(street_address).strip())
    
    # Add city if available
    if pd.notna(city) and str(city).strip():
        components.append(str(city).strip())
    
    # Add state and zip code together if available
    state_zip = []
    if pd.notna(state) and str(state).strip():
        state_zip.append(str(state).strip())
    if pd.notna(zip_code) and str(zip_code).strip():
        state_zip.append(str(zip_code).strip())
    
    if state_zip:
        components.append(" ".join(state_zip))
    
    # Join with commas for proper address formatting
    return ", ".join(components) if components else ""

def geocode_address_improved(street_address, city=None, zip_code=None, state=None, region="us") -> dict:
    """
    Enhanced geocode function that creates comprehensive addresses using all available components
    Returns a dictionary with geocoding results
    """
    # Create comprehensive address
    full_address = create_comprehensive_address(street_address, city, zip_code, state)
    
    if not full_address:
        return {"geo_status": "EMPTY", "input_address": ""}
    
    # Check cache first
    if full_address in _cache:
        result = _cache[full_address].copy()
        result["input_address"] = full_address
        return result

    params = {
        "address": full_address,
        "key": API_KEY,
        "region": region,
        "components": f"country:US"  # Restrict to US to improve accuracy
    }
    
    try:
        resp = session.get(GEOCODE_URL, params=params, timeout=15)
        resp.raise_for_status()
        data = resp.json()
    except Exception as e:
        result = {"geo_status": f"REQUEST_ERROR: {e}", "input_address": full_address}
        _cache[full_address] = result
        return result

    status = data.get("status", "UNKNOWN")
    if status == "OK" and data.get("results"):
        r0 = data["results"][0]
        result = {
            "geo_status": status,
            "input_address": full_address,
            "formatted_address": r0.get("formatted_address"),
            "lat": r0.get("geometry", {}).get("location", {}).get("lat"),
            "lng": r0.get("geometry", {}).get("location", {}).get("lng"),
            "place_id": r0.get("place_id"),
            "types": ",".join(r0.get("types", [])),
            "location_type": r0.get("geometry", {}).get("location_type"),  # Added location precision info
            "address_components_count": len(r0.get("address_components", []))  # Number of matched components
        }
    else:
        result = {
            "geo_status": status, 
            "input_address": full_address,
            "error_message": data.get("error_message")
        }

    _cache[full_address] = result
    time.sleep(0.2)  # Rate limiting
    return result

# Test the improved function with a sample
print("Testing improved geocoding function:")
sample_row = proper_addresses.iloc[0]
print(f"Sample address components:")
print(f"  Street: {sample_row['address_standardized_OFF_parenthesis']}")
print(f"  City: {sample_row['city']}")
print(f"  Zip: {sample_row['zip_code']}")
print(f"  State: {sample_row['state']}")

result = geocode_address_improved(
    sample_row['address_standardized_OFF_parenthesis'],
    sample_row['city'],
    sample_row['zip_code'],
    sample_row['state']
)

print(f"\nImproved geocoding result:")
for key, value in result.items():
    print(f"  {key}: {value}")

Testing improved geocoding function:
Sample address components:
  Street: 1590 US 40 S
  City: Heber City
  Zip: 84032
  State: UT

Improved geocoding result:
  geo_status: OK
  input_address: 1590 US 40 S, Heber City, UT 84032
  formatted_address: 1590 US-40, Heber City, UT 84032, USA
  lat: 40.4878994
  lng: -111.4084917
  place_id: ChIJmVOuQLf1TYcR9iVDdulusIM
  types: premise,street_address
  location_type: GEOMETRIC_CENTER
  address_components_count: 8
  partial_match: False


In [29]:
df[df['Address_Type'] == 'Proper']

,clean_line1,clean_line2,line3,city,zip_code,label,phone,year,major_city,state,...,min_distance_miles_unconstrained,min_distance_sources_unconstrained,Mid_Lat,Mid_Long,Match_Comments,Manual_Match,Manual_Lat,Manual_Long,Final_Lat,Final_Long
8,"Heber City , 84032 K & T's Last Stop ( Chevron )",435-654-4642 1590 US 40 S ( 1/2 mi S of jct US...,) <U+2610> 0 <U+2610> <U+2610>,Heber City,84032,K & T's Last Stop ( Chevron ),435-654-4642,2008,Heber City,UT,...,0.024074,Webscraped vs YellowPages (Phone),40.487595,-111.408021,NaN,NaN,NaN,NaN,40.487595,-111.408021
9,Heber City ( 84032 ) Silver Eagle Country Store,435-654-4642 1590 US 40 S ( 1/2 mi S of jct US...,189 ) <U+25A1> <U+2610><U+2610> <U+25A1>,Heber City,84032,Silver Eagle Country Store,435-654-4642,2014,Heber City,UT,...,0.024074,Webscraped vs YellowPages (Phone),40.487595,-111.408021,NaN,NaN,NaN,NaN,40.487595,-111.408021
10,E Heber City ( 84032 ) Silver Eagle Country Store,435-654-4642 1590 US 40 S ( 1/2 mi S of jct US...,<U+2610><U+2610>,Heber City,84032,Silver Eagle Country Store,435-654-4642,2015,Heber City,UT,...,0.024074,Webscraped vs YellowPages (Phone),40.487595,-111.408021,NaN,NaN,NaN,NaN,40.487595,-111.408021
11,Heber City ( 84032 ) Silver Eagle Country Store,435-654-4642 1590 US 40 S ( 1/2 mi S of jct US...,<U+2610> D,Heber City,84032,Silver Eagle Country Store,435-654-4642,2016,Heber City,UT,...,0.024074,Webscraped vs YellowPages (Phone),40.487595,-111.408021,NaN,NaN,NaN,NaN,40.487595,-111.408021
27,Roosevelt ( 84066 ) Top Stop # 66 ( Chevron ),435-722-9940 1128 US 40 ( 1 mi W of town ),<U+25A1> S <U+25A1>,Roosevelt,84066,Top Stop # 66 ( Chevron ),435-722-9940,2014,Roosevelt,UT,...,0.018445,Webscraped vs YellowPages (Phone),40.283397,-110.011284,NaN,NaN,NaN,NaN,40.283397,-110.011284
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2276,"B Eureka , 95503 Golden Gate Petroleum ( Texac...",707-443-1891 3505 US 101 & Highland,HRS 000,Eureka,95503,Golden Gate Petroleum ( Texaco ) ),707-443-1891,2006,Eureka,CA,...,0.000991,Yelp (Phone) vs YellowPages (Phone),40.774982,-124.190246,NaN,NaN,NaN,NaN,40.774982,-124.190246
2277,"Eureka , 95503 Golden Gate Petroleum ( Texaco )",707-443-1891 3505 US 101 & Highland,000 000,Eureka,95503,Golden Gate Petroleum ( Texaco ),707-443-1891,2007,Eureka,CA,...,0.000991,Yelp (Phone) vs YellowPages (Phone),40.774982,-124.190246,NaN,NaN,NaN,NaN,40.774982,-124.190246
2278,"B Eureka , 95503 Golden Gate Petroleum ( Texaco )",707-443-1891 3505 US 101 & Highland,24 HRS <U+25A1> <U+25A1>,Eureka,95503,Golden Gate Petroleum ( Texaco ),707-443-1891,2008,Eureka,CA,...,0.000991,Yelp (Phone) vs YellowPages (Phone),40.774982,-124.190246,NaN,NaN,NaN,NaN,40.774982,-124.190246
2279,B Eureka ( 95503 ) Texaco Star,707-443-1891 3505 US 101 & Highland,24 O,Eureka,95503,Texaco Star,707-443-1891,2015,Eureka,CA,...,0.000991,Yelp (Phone) vs YellowPages (Phone),40.774982,-124.190246,NaN,NaN,NaN,NaN,40.774982,-124.190246


In [30]:
# Set up Google Geocoding API
API_KEY = os.getenv('GOOGLE_MAPS_API_KEY')
if not API_KEY:
    raise RuntimeError("GOOGLE_MAPS_API_KEY not found in environment variables")

GEOCODE_URL = "https://maps.googleapis.com/maps/api/geocode/json"
print(f"API Key loaded: {API_KEY[:10]}...{API_KEY[-4:]}")  # Show partial key for verification

API Key loaded: AIzaSyBRG3...wIqk


In [31]:
# Create session and cache for API requests
session = requests.Session()
_cache = {}

def geocode_address(address: str, region: str = "us") -> dict:
    """
    Geocode an address using Google Maps Geocoding API
    Returns a dictionary with geocoding results
    """
    if not isinstance(address, str) or not address.strip():
        return {"geo_status": "EMPTY"}
    
    if address in _cache:
        return _cache[address]

    params = {
        "address": address,
        "key": API_KEY,
        "region": region  # bias results by country
    }
    
    try:
        resp = session.get(GEOCODE_URL, params=params, timeout=15)
        resp.raise_for_status()
        data = resp.json()
    except Exception as e:
        result = {"geo_status": f"REQUEST_ERROR: {e}"}
        _cache[address] = result
        return result

    status = data.get("status", "UNKNOWN")
    if status == "OK" and data.get("results"):
        r0 = data["results"][0]
        result = {
            "geo_status": status,
            "formatted_address": r0.get("formatted_address"),
            "lat": r0.get("geometry", {}).get("location", {}).get("lat"),
            "lng": r0.get("geometry", {}).get("location", {}).get("lng"),
            "place_id": r0.get("place_id"),
            "types": ",".join(r0.get("types", [])),
        }
    else:
        # Common statuses: ZERO_RESULTS, OVER_QUERY_LIMIT, REQUEST_DENIED, INVALID_REQUEST
        result = {"geo_status": status, "error_message": data.get("error_message")}

    _cache[address] = result
    time.sleep(0.2)  # gentle throttle to avoid QPS issues
    return result

In [32]:
# Check if the address column exists in the full dataset
addr_col = "address_standardized_OFF_parenthesis"
if addr_col not in df.columns:
    print(f"Available columns: {list(df.columns)}")
    raise KeyError(f"Column '{addr_col}' not found in the CSV.")

# Count addresses to geocode
proper_mask = df['Address_Type'] == 'Proper'
proper_count = proper_mask.sum()

print(f"Total rows in dataset: {len(df)}")
print(f"Found {proper_count} 'Proper' addresses to geocode")
print(f"Sample addresses from {addr_col}:")
print(df.loc[proper_mask, addr_col].head())

# Show the first few rows to verify
df.head()

Total rows in dataset: 2333
Found 297 'Proper' addresses to geocode
Sample addresses from address_standardized_OFF_parenthesis:
8     1590 US 40 S
9     1590 US 40 S
10    1590 US 40 S
11    1590 US 40 S
27      1128 US 40
Name: address_standardized_OFF_parenthesis, dtype: object


,clean_line1,clean_line2,line3,city,zip_code,label,phone,year,major_city,state,...,min_distance_miles_unconstrained,min_distance_sources_unconstrained,Mid_Lat,Mid_Long,Match_Comments,Manual_Match,Manual_Lat,Manual_Long,Final_Lat,Final_Long
0,"Coalville , 84017 Holiday Hills ( 66 )",435-336-4421 I-80 Exit 162 ( UT 280 ),MO,Coalville,84017,Holiday Hills ( 66 ),435-336-4421,2006,Coalville,UT,...,0.071012,Webscraped vs Yelp (Phone),40.910948,-111.404529,NaN,NaN,NaN,NaN,40.910948,-111.404529
1,"Coalville , 84017 Hills ( 66 )",435-336-4421 I-80 Holiday Exit 162 ( UT 280 ),M <U+25A1>,Coalville,84017,Hills ( 66 ),435-336-4421,2007,Coalville,UT,...,0.071012,Webscraped vs Yelp (Phone),40.910948,-111.404529,NaN,NaN,NaN,NaN,40.910948,-111.404529
2,"D Coalville , 84017 Holiday Hills ( 66 ) )",4 435-336-4421 I-80 Exit 162 ( UT 280 ),M <U+2610> <U+2610> <U+2610>,Coalville,84017,Holiday Hills ( 66 ) ),435-336-4421,2008,Coalville,UT,...,0.071012,Webscraped vs Yelp (Phone),40.910948,-111.404529,NaN,NaN,NaN,NaN,40.910948,-111.404529
3,D Coalville ( 84017 ) Holiday Hills ( 66 ),435-336-4421 I-80 Exit 162 ( UT 280 ),M <U+25A1> <U+2610>,Coalville,84017,Holiday Hills ( 66 ),435-336-4421,2014,Coalville,UT,...,0.071012,Webscraped vs Yelp (Phone),40.910948,-111.404529,NaN,NaN,NaN,NaN,40.910948,-111.404529
4,D Coalville ( 84017 ) Holiday Hills ( 66 ),4 435-336-4421 I-80 Exit 162 ( UT 280 ),D M <U+25A1> D D,Coalville,84017,Holiday Hills ( 66 ),435-336-4421,2015,Coalville,UT,...,0.071012,Webscraped vs Yelp (Phone),40.910948,-111.404529,NaN,NaN,NaN,NaN,40.910948,-111.404529


In [33]:
# Test with one random address first
import random

# Get a random address from the proper addresses
proper_addresses_series = df.loc[df['Address_Type'] == 'Proper', addr_col].dropna()
test_address = proper_addresses_series.sample(1).iloc[0]
print(f"Testing with address: {test_address}")

# Test the geocoding function
result = geocode_address(test_address)
print(f"\nGeocoding result:")
for key, value in result.items():
    print(f"  {key}: {value}")
    
# Check if it was successful
if result.get('geo_status') == 'OK':
    print(f"\n✅ Success! Found coordinates: ({result['lat']}, {result['lng']})")
else:
    print(f"\n❌ Issue: {result.get('geo_status', 'Unknown error')}")
    if 'error_message' in result:
        print(f"Error message: {result['error_message']}")

Testing with address: 380 US 40-191 S Pint

Geocoding result:
  geo_status: OK
  formatted_address: 380 US-40 191 s, Carneys Point, NJ 08069, USA
  lat: 39.6800657
  lng: -75.471584
  place_id: Ei0zODAgVVMtNDAgMTkxIHMsIENhcm5leXMgUG9pbnQsIE5KIDA4MDY5LCBVU0EiIRofChYKFAoSCUPEz7mtHceJEdY19Orw8IK3EgUxOTEgcw
  types: street_address,subpremise

✅ Success! Found coordinates: (39.6800657, -75.471584)

Geocoding result:
  geo_status: OK
  formatted_address: 380 US-40 191 s, Carneys Point, NJ 08069, USA
  lat: 39.6800657
  lng: -75.471584
  place_id: Ei0zODAgVVMtNDAgMTkxIHMsIENhcm5leXMgUG9pbnQsIE5KIDA4MDY5LCBVU0EiIRofChYKFAoSCUPEz7mtHceJEdY19Orw8IK3EgUxOTEgcw
  types: street_address,subpremise

✅ Success! Found coordinates: (39.6800657, -75.471584)


In [34]:
# ============================================================================
# IMPROVED GEOCODING WITH COMPREHENSIVE ADDRESS CONSTRUCTION
# ============================================================================

import pandas as pd
import os
import time
import requests
from dotenv import load_dotenv
from geopy.distance import geodesic

# Load environment variables and data
load_dotenv(r'C:\Users\clint\Desktop\Geocoding_Documentation\.env')
df = pd.read_csv(r'C:\Users\clint\Desktop\Geocoding_Documentation\Notebooks\9.csv')

# Set up Google Geocoding API
API_KEY = os.getenv('GOOGLE_MAPS_API_KEY')
if not API_KEY:
    raise RuntimeError("GOOGLE_MAPS_API_KEY not found in environment variables")

GEOCODE_URL = "https://maps.googleapis.com/maps/api/geocode/json"
session = requests.Session()
_cache = {}

print(f"Loaded {len(df)} records")
print(f"Found {(df['Address_Type'] == 'Proper').sum()} 'Proper' addresses to geocode")

def create_comprehensive_address(street_address, city, zip_code, state):
    """
    Create a comprehensive address string by combining street address, city, zip code, and state
    This follows standard US address formatting for better geocoding results
    """
    components = []
    
    # Add street address if available
    if pd.notna(street_address) and str(street_address).strip():
        components.append(str(street_address).strip())
    
    # Add city if available
    if pd.notna(city) and str(city).strip():
        components.append(str(city).strip())
    
    # Add state and zip code together if available
    state_zip = []
    if pd.notna(state) and str(state).strip():
        state_zip.append(str(state).strip())
    if pd.notna(zip_code) and str(zip_code).strip():
        state_zip.append(str(zip_code).strip())
    
    if state_zip:
        components.append(" ".join(state_zip))
    
    # Join with commas for proper address formatting
    return ", ".join(components) if components else ""

def geocode_address_improved(street_address, city=None, zip_code=None, state=None, region="us") -> dict:
    """
    Enhanced geocode function that creates comprehensive addresses using all available components
    Returns a dictionary with geocoding results including additional metadata
    """
    # Create comprehensive address
    full_address = create_comprehensive_address(street_address, city, zip_code, state)
    
    if not full_address:
        return {"geo_status": "EMPTY", "input_address": ""}
    
    # Check cache first
    if full_address in _cache:
        result = _cache[full_address].copy()
        result["input_address"] = full_address
        return result

    params = {
        "address": full_address,
        "key": API_KEY,
        "region": region,
        "components": f"country:US"  # Restrict to US to improve accuracy
    }
    
    try:
        resp = session.get(GEOCODE_URL, params=params, timeout=15)
        resp.raise_for_status()
        data = resp.json()
    except Exception as e:
        result = {"geo_status": f"REQUEST_ERROR: {e}", "input_address": full_address}
        _cache[full_address] = result
        return result

    status = data.get("status", "UNKNOWN")
    if status == "OK" and data.get("results"):
        r0 = data["results"][0]
        result = {
            "geo_status": status,
            "input_address": full_address,
            "formatted_address": r0.get("formatted_address"),
            "lat": r0.get("geometry", {}).get("location", {}).get("lat"),
            "lng": r0.get("geometry", {}).get("location", {}).get("lng"),
            "place_id": r0.get("place_id"),
            "types": ",".join(r0.get("types", [])),
            "location_type": r0.get("geometry", {}).get("location_type"),  # ROOFTOP, RANGE_INTERPOLATED, etc.
            "address_components_count": len(r0.get("address_components", [])),  # Number of matched components
            "partial_match": r0.get("partial_match", False)  # Whether this was a partial match
        }
    else:
        result = {
            "geo_status": status, 
            "input_address": full_address,
            "error_message": data.get("error_message")
        }

    _cache[full_address] = result
    time.sleep(0.2)  # Rate limiting
    return result

# Original simple geocoding function for comparison
def geocode_address_original(address: str, region: str = "us") -> dict:
    """Original geocoding function using only the street address"""
    if not isinstance(address, str) or not address.strip():
        return {"geo_status": "EMPTY"}
    
    cache_key = f"original_{address}"
    if cache_key in _cache:
        return _cache[cache_key]

    params = {
        "address": address,
        "key": API_KEY,
        "region": region
    }
    
    try:
        resp = session.get(GEOCODE_URL, params=params, timeout=15)
        resp.raise_for_status()
        data = resp.json()
    except Exception as e:
        result = {"geo_status": f"REQUEST_ERROR: {e}"}
        _cache[cache_key] = result
        return result

    status = data.get("status", "UNKNOWN")
    if status == "OK" and data.get("results"):
        r0 = data["results"][0]
        result = {
            "geo_status": status,
            "formatted_address": r0.get("formatted_address"),
            "lat": r0.get("geometry", {}).get("location", {}).get("lat"),
            "lng": r0.get("geometry", {}).get("location", {}).get("lng"),
            "place_id": r0.get("place_id"),
            "types": ",".join(r0.get("types", [])),
        }
    else:
        result = {"geo_status": status, "error_message": data.get("error_message")}

    _cache[cache_key] = result
    time.sleep(0.2)
    return result

print("✅ Improved geocoding functions loaded successfully!")

Loaded 2333 records
Found 297 'Proper' addresses to geocode
✅ Improved geocoding functions loaded successfully!


In [35]:
# ============================================================================
# ENHANCED GEOCODING WITH STRICT STATE RESTRICTIONS (ZIPCODE RESTRICTION REMOVED)
# ============================================================================

def geocode_address_strict(street_address, city=None, zip_code=None, state=None, region="us") -> dict:
    """
    Enhanced geocode function with strict state filtering only
    Uses Google's component filtering to restrict results to specific state
    Zipcode restriction has been removed to allow more flexible matching
    """
    # Create comprehensive address
    full_address = create_comprehensive_address(street_address, city, zip_code, state)
    
    if not full_address:
        return {"geo_status": "EMPTY", "input_address": ""}
    
    # Create cache key that includes only state restrictions
    cache_key = f"strict_state_only_{full_address}_{state}"
    if cache_key in _cache:
        result = _cache[cache_key].copy()
        result["input_address"] = full_address
        return result

    # Build component restrictions - ONLY STATE AND COUNTRY
    components = ["country:US"]
    
    # Add state restriction if available
    if pd.notna(state) and str(state).strip():
        state_code = str(state).strip().upper()
        components.append(f"administrative_area_level_1:{state_code}")
    
    # ZIPCODE RESTRICTION REMOVED - No longer adding postal_code component

    params = {
        "address": full_address,
        "key": API_KEY,
        "region": region,
        "components": "|".join(components)  # Join multiple components with |
    }
    
    try:
        resp = session.get(GEOCODE_URL, params=params, timeout=15)
        resp.raise_for_status()
        data = resp.json()
    except Exception as e:
        result = {"geo_status": f"REQUEST_ERROR: {e}", "input_address": full_address}
        _cache[cache_key] = result
        return result

    status = data.get("status", "UNKNOWN")
    if status == "OK" and data.get("results"):
        r0 = data["results"][0]
        
        # Extract state from result to verify it matches our expectation
        result_state = None
        result_zip = None
        for component in r0.get("address_components", []):
            types = component.get("types", [])
            if "administrative_area_level_1" in types:
                result_state = component.get("short_name")
            elif "postal_code" in types:
                result_zip = component.get("short_name")
        
        result = {
            "geo_status": status,
            "input_address": full_address,
            "formatted_address": r0.get("formatted_address"),
            "lat": r0.get("geometry", {}).get("location", {}).get("lat"),
            "lng": r0.get("geometry", {}).get("location", {}).get("lng"),
            "place_id": r0.get("place_id"),
            "types": ",".join(r0.get("types", [])),
            "location_type": r0.get("geometry", {}).get("location_type"),
            "address_components_count": len(r0.get("address_components", [])),
            "partial_match": r0.get("partial_match", False),
            "result_state": result_state,  # State from geocoded result
            "result_zip": result_zip,      # Zip from geocoded result
            "input_state": str(state).strip().upper() if pd.notna(state) else None,
            "input_zip": str(zip_code).strip() if pd.notna(zip_code) else None,
            "state_match": result_state == str(state).strip().upper() if pd.notna(state) and result_state else None,
            "components_used": "|".join(components)
        }
    else:
        result = {
            "geo_status": status, 
            "input_address": full_address,
            "error_message": data.get("error_message"),
            "components_used": "|".join(components)
        }

    _cache[cache_key] = result
    time.sleep(0.2)  # Rate limiting
    return result

def validate_geocoding_results(df_results, state_col='input_state', result_state_col='result_state'):
    """
    Analyze geocoding results to identify state mismatches
    """
    # Get successful geocodes
    successful = df_results[df_results['geo_status'] == 'OK']
    
    if len(successful) == 0:
        print("No successful geocodes to analyze")
        return
    
    # Check state matches
    state_mismatches = successful[
        (successful[state_col].notna()) & 
        (successful[result_state_col].notna()) &
        (successful[state_col] != successful[result_state_col])
    ]
    
    print(f"Geocoding Results Analysis:")
    print(f"  Total successful geocodes: {len(successful)}")
    print(f"  State mismatches: {len(state_mismatches)} ({len(state_mismatches)/len(successful)*100:.1f}%)")
    
    if len(state_mismatches) > 0:
        print(f"\nSample state mismatches:")
        mismatch_cols = ['input_address', state_col, result_state_col, 'formatted_address']
        print(state_mismatches[mismatch_cols].head())
        
        # Show state mismatch breakdown
        mismatch_summary = state_mismatches.groupby([state_col, result_state_col]).size().head(10)
        print(f"\nTop state mismatch patterns:")
        for (input_state, result_state), count in mismatch_summary.items():
            print(f"  {input_state} → {result_state}: {count} cases")

print("✅ Enhanced geocoding functions with strict STATE-ONLY restrictions loaded!")
print("   🔄 Zipcode restriction has been REMOVED for more flexible matching")

✅ Enhanced geocoding functions with strict STATE-ONLY restrictions loaded!
   🔄 Zipcode restriction has been REMOVED for more flexible matching


In [36]:
# ============================================================================
# RUN STRICT GEOCODING ON ALL 'PROPER' ADDRESSES (STATE-ONLY RESTRICTIONS)
# ============================================================================

# Create a copy for strict geocoding results
df_strict = df.copy()

# Initialize new columns for strict geocoding results
strict_columns = ['geo_status', 'input_address', 'formatted_address', 'lat', 'lng', 
                 'place_id', 'types', 'location_type', 'address_components_count', 
                 'partial_match', 'result_state', 'result_zip', 'input_state', 
                 'input_zip', 'state_match', 'components_used']

for col in strict_columns:
    df_strict[f'Strict_Geocoding_{col}'] = None

print(f"Starting strict geocoding process on {len(proper_indices)} addresses...")
print("This will use STATE-ONLY restrictions (zipcode restriction removed) to improve accuracy...")

start_time = time.time()

# Process each proper address with strict geocoding
for i, idx in enumerate(proper_indices):
    if i % 25 == 0:  # Progress update every 25 addresses
        elapsed = time.time() - start_time
        rate = i / elapsed if elapsed > 0 else 0
        remaining = len(proper_indices) - i
        eta = remaining / rate if rate > 0 else 0
        print(f"Progress: {i}/{len(proper_indices)} addresses processed... "
              f"Rate: {rate:.1f}/min, ETA: {eta/60:.1f} min")
    
    row = df.loc[idx]
    
    # Use strict geocoding with STATE-ONLY restrictions
    result = geocode_address_strict(
        row['address_standardized_OFF_parenthesis'],
        row['city'],
        row['zip_code'],
        row['state']
    )
    
    # Store results in the dataframe
    for col in strict_columns:
        df_strict.loc[idx, f'Strict_Geocoding_{col}'] = result.get(col)

elapsed_total = time.time() - start_time
print(f"\n✅ Strict geocoding complete!")
print(f"Total time: {elapsed_total/60:.1f} minutes")
print(f"Average rate: {len(proper_indices)/(elapsed_total/60):.1f} addresses/minute")

# Analyze results
strict_success_count = (df_strict['Strict_Geocoding_geo_status'] == 'OK').sum()
print(f"\nStrict geocoding results:")
print(f"Successful geocodes: {strict_success_count}/{len(proper_indices)} ({strict_success_count/len(proper_indices)*100:.1f}%)")

# Show status breakdown
strict_status_counts = df_strict['Strict_Geocoding_geo_status'].value_counts()
print(f"\nStatus breakdown:")
for status, count in strict_status_counts.items():
    if pd.notna(status):
        print(f"  {status}: {count}")

# Analyze state matching
successful_strict = df_strict[df_strict['Strict_Geocoding_geo_status'] == 'OK']
if len(successful_strict) > 0:
    # Count state matches
    state_matches = successful_strict['Strict_Geocoding_state_match'] == True
    state_match_count = state_matches.sum()
    
    print(f"\nState Matching Analysis:")
    print(f"  Addresses with state data: {successful_strict['Strict_Geocoding_input_state'].notna().sum()}")
    print(f"  Perfect state matches: {state_match_count}")
    print(f"  State match rate: {state_match_count/len(successful_strict)*100:.1f}%")
    
    # Show mismatches
    mismatches = successful_strict[successful_strict['Strict_Geocoding_state_match'] == False]
    if len(mismatches) > 0:
        print(f"\nState mismatches found: {len(mismatches)}")
        mismatch_summary = mismatches.groupby(['Strict_Geocoding_input_state', 'Strict_Geocoding_result_state']).size().head(5)
        print("Top mismatch patterns:")
        for (input_state, result_state), count in mismatch_summary.items():
            print(f"  {input_state} → {result_state}: {count} cases")

# Save results with the requested filename
output_file_strict = r'C:\Users\clint\Desktop\Geocoding_Documentation\Notebooks\9_5_strict_improved.csv'
df_strict.to_csv(output_file_strict, index=False)
print(f"\n💾 Strict geocoding results saved to: {output_file_strict}")
print(f"📋 File contains {len(df_strict)} total rows with state-only geocoding restrictions applied")

Starting strict geocoding process on 297 addresses...
This will use STATE-ONLY restrictions (zipcode restriction removed) to improve accuracy...
Progress: 0/297 addresses processed... Rate: 0.0/min, ETA: 0.0 min
Progress: 25/297 addresses processed... Rate: 9.4/min, ETA: 0.5 min
Progress: 25/297 addresses processed... Rate: 9.4/min, ETA: 0.5 min
Progress: 50/297 addresses processed... Rate: 9.8/min, ETA: 0.4 min
Progress: 50/297 addresses processed... Rate: 9.8/min, ETA: 0.4 min
Progress: 75/297 addresses processed... Rate: 10.9/min, ETA: 0.3 min
Progress: 75/297 addresses processed... Rate: 10.9/min, ETA: 0.3 min
Progress: 100/297 addresses processed... Rate: 10.7/min, ETA: 0.3 min
Progress: 100/297 addresses processed... Rate: 10.7/min, ETA: 0.3 min
Progress: 125/297 addresses processed... Rate: 11.0/min, ETA: 0.3 min
Progress: 125/297 addresses processed... Rate: 11.0/min, ETA: 0.3 min
Progress: 150/297 addresses processed... Rate: 11.9/min, ETA: 0.2 min
Progress: 150/297 addresses 